# Air Quality Data Collection
This notebook will be used to get the air quality data and clean the datasets, so we can use them for data analysis and making our model. The data will be collected using the dustmonitor API of TNO (https://ilm2.site.dustmonitoring.nl). 

In [41]:
import pandas as pd
import os
import requests
import warnings

from datetime import date
from dateutil.relativedelta import relativedelta

pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')

print('Pandas version:', pd.__version__)

Pandas version: 2.2.3


In [42]:
# make new directory for data
root_dir = os.path.join("data")
os.makedirs(root_dir, exist_ok=True)

# save data from api
data_dir = os.path.join(root_dir, 'api_data')
os.makedirs(data_dir, exist_ok=True)

Firts we will collect all the data we want from the dust monitoring API. We want to collect as much data starting in 2021, because that is when most sensors on the locations we have selected started collecting data. The data will be collected with a granularity of 1 day, with the average value of the day. The reason we have selected certain locations is because they are located in the centre of Eindhoven, near a green area or outside the centre.

In [43]:
def get_data(start: str, end: str, interval: int):
    data = requests.get(
        f"https://ilm2.site.dustmonitoring.nl/download?from={start}&to={end}&interval={interval}&align=1&type=csv-semicolon&p=545&p=535&p=522&p=547&p=551&p=527&p=528&s=10&s=11&s=128&s=129&s=130&s=145&s=146"
    )

    return data.text

_date = date(2021, 1, 1)

while True:
    start_date = _date
    end_date = _date + relativedelta(months=12)
    interval = 86400
    data = get_data(start_date.strftime("%Y-%m-%d"), end_date.strftime("%Y-%m-%d"), interval)

    file_path = os.path.join(data_dir, f"data_{start_date.strftime('%Y-%m-%d')}_{end_date.strftime('%Y-%m-%d')}.csv")

    with open(file_path, "+w") as f:
        f.write(data)

    _date = end_date
    if _date > date(2024, 1, 2):
        break

The data needs to be formatted correctly so it is usable, we will remove the headers of the csv and set the correct columns.

In [97]:
csv_dir = os.path.join(root_dir, "csv")
os.makedirs(csv_dir, exist_ok=True)

dfs = []
for file in os.listdir(data_dir):
    file_path = os.path.join(data_dir, file)

    if not os.path.isfile(file_path):
        continue

    df = pd.read_csv(file_path, index_col=False, sep=";")

    header_string = df.iloc[:2].values
    row_1 = [row.split(".")[0] for row in df.columns.tolist()]
    row_2 = header_string[0]

    new_columns = []
    for row1, row2 in zip(row_1, row_2):
        row1 = row1.replace("Unnamed: ", "")
        new_columns.append(f"{row1}-{row2}")
    
    df = df.iloc[2:]

    df.columns = new_columns

    csv_file_path = os.path.join(csv_dir, file)
    df.to_csv(csv_file_path, index=False, index_label=False)

In [98]:
df = []
for file in os.listdir(csv_dir):
    csv_file_path = os.path.join(csv_dir, file)
    df = pd.read_csv(csv_file_path, index_col=False)
    dfs.append(df)

df = pd.concat(dfs, ignore_index=True)
df.shape

(1425, 44)

In [99]:
print(df.columns.tolist())

['ï»¿-Tijd', '1-Tijd', 'I07-Lat', 'I07-Lon', 'I07-PM1', 'I07-PM2.5', 'I07-PM10', 'I07-NO2', 'I12-Lat', 'I12-Lon', 'I12-PM1', 'I12-PM2.5', 'I12-PM10', 'I12-NO2', 'I17-Lat', 'I17-Lon', 'I17-PM1', 'I17-PM2.5', 'I17-PM10', 'I17-NO2', 'I19-Lat', 'I19-Lon', 'I19-PM1', 'I19-PM2.5', 'I19-PM10', 'I19-NO2', 'I30-Lat', 'I30-Lon', 'I30-PM1', 'I30-PM2.5', 'I30-PM10', 'I30-NO2', 'I36-Lat', 'I36-Lon', 'I36-PM1', 'I36-PM2.5', 'I36-PM10', 'I36-NO2', 'I37-Lat', 'I37-Lon', 'I37-PM1', 'I37-PM2.5', 'I37-PM10', 'I37-NO2']


Next we will split the dataframe by location, saving each location to a new csv. We will also drop the first column that contains the UTC time info, as we will only use the local time. The data will also be sorted by chronological order.

In [100]:
location_dir = os.path.join(root_dir, "location")
os.makedirs(location_dir, exist_ok=True)

for location in range(1, 40):
    location = str(location)
    if len(location) < 2:
        location = '0' + location
    
    location_columns = df.columns[:2].to_list()
    for col in df.columns:
        if col.startswith(f'I{location}'):
            location_columns.append(col)

    if len(location_columns) > 2:
        df_temp = df[location_columns]
        df_temp = df_temp.drop(['ï»¿-Tijd'], axis=1, errors='ignore')
        df_temp['1-Tijd'] = pd.to_datetime(df_temp['1-Tijd'])
        df_temp = df_temp.sort_values(by='1-Tijd')

        location_csv_path = os.path.join(location_dir, f"I{location}.csv")
        df_temp.to_csv(location_csv_path, index=False, index_label=False)

In [101]:
# sample of one of the location datasets
df_test = pd.read_csv('./data/location/I07.csv')
df_test.sample(15)

,1-Tijd,I07-Lat,I07-Lon,I07-PM1,I07-PM2.5,I07-PM10,I07-NO2
76,2021-03-18 01:00:00,"51,4568","5,4382","4,75","5,41","9,10",17.0
86,2021-03-28 01:00:00,"51,4568","5,4383","4,81","7,17","14,27",22.0
101,2021-04-12 02:00:00,"51,4569","5,4383","2,03","2,73","6,66",NaN
16,2021-01-17 01:00:00,"51,4568","5,4383","19,48","20,99","23,84",28.0
768,2023-02-04 01:00:00,"51,4569","5,4384","5,56","8,05","15,40",15.0
1386,2024-10-12 02:00:00,"51,4567","5,4381","4,06","4,99","8,96",23.0
895,2023-06-11 02:00:00,"51,4568","5,4384","6,87","7,70","11,59",14.0
1302,2024-07-20 02:00:00,"51,4568","5,4383","2,03","2,57","6,66",12.0
1266,2024-06-14 02:00:00,"51,4569","5,4384","2,08","2,76","6,68",17.0
458,2022-04-02 02:00:00,"51,4568","5,4383","4,06","4,71","7,81",14.0
